In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import display, Image 
from langgraph.types import Send
from langgraph.graph.message import add_messages
from typing import Sequence
from langchain_core.messages import BaseMessage, HumanMessage
from typing import Annotated


class PersonInfo(TypedDict):
    voice_id: int
    first_name: str
    last_name: str
    full_name: str

class WorkflowState(TypedDict):
    person_data: list[PersonInfo]
    messages: Annotated[Sequence[BaseMessage], add_messages]

class ProcessState(TypedDict):
    person: PersonInfo
    
def generate_full_name(state: ProcessState):
    person = state["person"]
    voice_id = person["voice_id"]
    first_name = person.get("first_name", "")
    last_name = person.get("last_name", "")
    full_name = f"{first_name}, {last_name}".strip(", ")

    message = HumanMessage(
        content="",
        additional_kwargs={"voice_id": voice_id, "full_message": full_name}
    )
    return {"messages": [message]}

def distribute_tasks(state: WorkflowState):
    return [Send("generate_full_name", {"person": person}) for person in state["person_data"]]

def update_full_names(state: WorkflowState):
    messages = state["messages"]
    people = state["person_data"]
    
    for person in people:
        voice_id = person["voice_id"]
        for msg in messages:
            if msg.additional_kwargs.get("voice_id") == voice_id:
                person["full_name"] = msg.additional_kwargs.get("full_message", "")
                break
                
    return {"person_data": people}

# Define Workflow
workflow = StateGraph(WorkflowState)
workflow.add_node("generate_full_name", generate_full_name)
workflow.add_node("update_full_names", update_full_names)

workflow.add_conditional_edges(START, distribute_tasks, ["generate_full_name"])
workflow.add_edge("generate_full_name", "update_full_names")
workflow.add_edge("update_full_names", END)
app = workflow.compile()

 

# Provide input with correct keys
response = app.invoke({
    "person_data": [
        {"voice_id": 0, "first_name": "Arman", "last_name": "Ashraf", "full_name": ""},
        {"voice_id": 1, "first_name": "Usman", "last_name": "Ali", "full_name": ""}
    ]
})

print(response)
